# Теорема Байеса

Этот блокнот является частью [Bite Size Bayes](https://allendowney.github.io/BiteSizeBayes/), введения в вероятность и байесовскую статистику с использованием Python.

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Законы%20вероятности.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

Следующая ячейка загружает файл `utils.py`, содержащий некоторую полезную функцию, которая нам понадобится:

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/BiteSizeBayes/raw/master/utils.py')

Следующая ячейка загружает файл данных, который мы будем использовать в этом блокноте:

In [2]:
download('https://github.com/AllenDowney/BiteSizeBayes/raw/master/gss_bayes.csv')

Если все, что нам нужно, установлено, следующая ячейка должна работать без сообщений об ошибках:

In [3]:
import pandas as pd
import numpy as np

from utils import values

## Обзор

[В предыдущем блокноте](https://dfedorov.spb.ru/pandas/downey/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD%D1%8B%20%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8.html) я определил *вероятность*, *конъюнкцию* и *условную вероятность* и использовал данные из GSS для вычисления вероятности различных логических утверждений.

Для обзора, вот как мы загрузили набор данных:

In [4]:
gss = pd.read_csv('gss_bayes.csv', index_col=0)

А вот и определенные нами логические утверждения, представленные с помощью логических серий (Boolean series):

In [5]:
banker = (gss['indus10'] == 6870)

In [6]:
female = (gss['sex'] == 2)

In [7]:
liberal = (gss['polviews'] < 4)

In [8]:
democrat = (gss['partyid'] <= 1)

Я определил следующую функцию, которая использует `mean` для вычисления доли значений `True` в логической серии:

In [9]:
def prob(A):
    """Computes the probability of a proposition, A.
    
    A: Boolean series
    
    returns: probability
    """
    assert isinstance(A, pd.Series)
    assert A.dtype == 'bool'
    
    return A.mean()

Итак, мы можем вычислить вероятность такого утверждения:

In [10]:
prob(female)

0.5378575776019476

Затем мы использовали оператор `&` для вычисления вероятности конъюнкции, например:

In [11]:
prob(female & banker)

0.011381618989653074

Затем я определил следующую функцию, которая использует оператор скобок для вычисления условной вероятности:

In [12]:
def conditional(A, B):
    """Conditional probability of A given B.
    
    A: Boolean series
    B: Boolean series
    
    returns: probability
    """
    return prob(A[B])

Мы показали, что конъюнкция коммутативна, так что `prob(A & B)` равно `prob(B & A)` для любых логических утверждений `A` и `B`.

Например:

In [13]:
prob(liberal & democrat)

0.1425238385067965

In [14]:
prob(democrat & liberal)

0.1425238385067965

Но условная вероятность *НЕ* коммутативна, поэтому `conditional(A, B)` обычно не то же самое, что `conditional(B, A)`.

Например, вот вероятность того, что респондент - женщина, учитывая, что это банкир.

In [15]:
conditional(female, banker)

0.7706043956043956

И вот вероятность того, что респондент - банкир, учитывая, что она женщина.

In [16]:
conditional(banker, female)

0.02116102749801969

Даже не близко.

## Другие утверждения

Для разнообразия наших примеров давайте определим некоторые новые утверждения.

Вот вероятность того, что случайный респондент - мужчина.

In [17]:
male = (gss['sex']==1)
prob(male)

0.46214242239805237

Отраслевой код для "Строительства" (Construction) - `770`. Назовем кого-нибудь из этой области "builder" (строителем).

In [18]:
builder = (gss['indus10'] == 770)
prob(builder)

0.05978900385473727

И давайте определимся с утверждениями для консерваторов и республиканцев:

In [19]:
conservative = (gss['polviews'] > 4)
prob(conservative)

0.3419354838709677

In [20]:
republican = (gss['partyid'].isin([5,6]))
prob(republican)

0.2610062893081761

Функция `isin` проверяет, находятся ли значения в заданной последовательности. 

В этом примере значения `5` и `6` представляют ответы "Сильный республиканец" (Strong Republican) и "Несильный республиканец" (Not Strong Republican).

Наконец, я буду использовать `age` для определения утверждений для `young` (молодой) и `old` (пожилой).

In [21]:
young = (gss['age'] < 30)
prob(young)

0.19435991073240008

In [22]:
old = (gss['age'] >= 65)
prob(old)

0.17328058429701765

Для этих порогов я выбрал круглые числа около 20-го и 80-го процентилей. В зависимости от вашего возраста вы можете соглашаться или не соглашаться с этими определениями `young` (молодой) и `old` (пожилой).

**Упражнение:** Есть [известная цитата](https://quoteinvestigator.com/2014/02/24/heart-head/) о молодых людях, стариках, либералах и консерваторах, которая звучит примерно так:

> Если в 25 вы не либерал, у вас нет сердца. Если в 35 лет вы не консерватор, у вас нет мозга.

Независимо от того, согласны вы с этим утверждением или нет, оно предполагает некоторые вероятности, которые мы можем вычислить в качестве проверки.

Используйте `prob` и `conditional` для вычисления этих вероятностей.

* Какова вероятность того, что случайно выбранный респондент окажется молодым либералом?

* Какова вероятность того, что молодой человек будет либералом?

* Какая доля респондентов - пожилые консерваторы?

* Какая часть консерваторов - люди старшего возраста?

Для каждого утверждения подумайте, выражает ли оно конъюнкцию, условную вероятность или и то, и другое.

А для условных вероятностей будьте осторожны с порядком!

In [23]:
# Решение здесь

Если ваш последний ответ больше 30%, значит, вы получили его наоборот!

## Вперед!

В этом ноутбуке мы выведем три отношения между конъюнкцией и условной вероятностью:

* **Теорема 1**. Использование конъюнкции для вычисления условной вероятности.

* **Теорема 2**: Использование условной вероятности для вычисления конъюнкции.

* **Теорема 3**: Использование `conditional(A, B)` для вычисления `conditional(B, A)`.

Теорема 3 также известна как *теорема Байеса*, которая является основой байесовской статистики.

В некоторых частях этого блокнота будет полезно использовать математические обозначения вероятностей, поэтому я представлю их сейчас.

* $P(A)$ - это вероятность утверждения $A$.

* $P(A~\mathrm{and}~B)$ - это вероятность конъюнкции $A$ и $B$, то есть вероятность того, что оба утверждения верны.

* $P(A | B)$ - это условная вероятность $A$ при условии, что $B$ истинно. Вертикальная линия между $A$ и $B$ произносится как "дано".

Теперь мы готовы к Теореме 1.

## Теорема 1

Какая часть строителей - мужчины? Мы уже видели один способ вычислить ответ:

1. с помощью оператора скобок выберите строителей, затем

2. используйте `mean`, чтобы вычислить долю строителей мужского пола.

Мы можем записать эти шаги так:

In [24]:
male[builder].mean()

0.8920936545639634

Или мы можем использовать функцию `conditional`, которая делает то же самое:

In [25]:
conditional(male, builder)

0.8920936545639634

Но есть другой способ: чтобы вычислить долю строителей-мужчин, мы можем вычислить отношение двух вероятностей:

1. долю респондентов строителей-мужчин и

2. долю респондентов строителей.

Вот как это выглядит:


In [26]:
prob(male & builder) / prob(builder)

0.8920936545639634

Результат тот же.

Этот пример демонстрирует *общее правило, которое связывает условную вероятность и конъюнкцию*. 

Вот как это выглядит в математической записи:

$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$

И это Теорема 1.

В этом примере:

`conditional(male, builder) = prob(male & builder) / prob(builder)`

**Упражнение:** Какая часть консерваторов - республиканцы? Вычислите ответ двумя способами:

* используйте функцию `conditional` (которая использует оператор скобки) и

* используйте Теорему 1.

Подтвердите, что вы получили такой же ответ.

*Примечание*: из-за арифметики с плавающей запятой результаты могут не совпадать, но почти все цифры должны совпадать.

In [27]:
# Решение здесь

## Доказательство?

На самом деле я не доказал Теорему 1; в основном, это утверждение о том, что означает условная вероятность.

Например, рассмотрим эту диаграмму Венна:

<img width="200" src="https://github.com/AllenDowney/BiteSizeBayes/raw/master/theorem1_venn_diagram.png">

Синий кружок представляет респондентов-мужчин. Красный кружок представляет строителей. На пересечении изображены мужчины-строители.

Чтобы вычислить долю мужчин-строителей, мы можем вычислить отношение пересечения, которое представляет собой `prob(male & builder)`, к красному кружку, то есть `prob(builder)`.

**Упражнение:** Для практики вычислите долю пожилых банкиров двумя способами: используя функцию `conditional` и Теорему 1.

In [28]:
# Решение здесь

## Теорема 2

Снова Теорема 1:

$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$ 

Если умножить обе части на $P(B)$, получим Теорему 2.

$P(A~\mathrm{and}~B) = P(B) P(A|B)$

Эта формула предлагает второй способ вычисления конъюнкции: вместо использования оператора `&` мы можем вычислить произведение двух вероятностей.

Посмотрим, сработает ли это для `conservative` (консерваторов) и `republican` (республиканцев). 

Вот результат с использованием `&`:

In [29]:
prob(conservative & republican)

0.15396632176912153

И вот результат использования Теоремы 2:

In [30]:
prob(republican) * conditional(conservative, republican)

0.1539663217691215

Из-за ошибок с плавающей запятой они могут не совпадать, но почти все цифры одинаковы.

**Упражнение:** Проверьте Теорему 2 еще раз, вычислив долю респондентов, которые являются пожилыми либералами двумя способами:

* с использованием оператора `&`, и

* используя Теорему 2.

Результаты должны быть такими же или, по крайней мере, очень близкими.

In [31]:
# Решение здесь

## Конъюнкция коммутативна

Мы уже установили, что конъюнкция коммутативна. В математической записи это означает:

$P(A~\mathrm{and}~B) = P(B~\mathrm{and}~A)$

Если применить Теорему 2 к обеим сторонам, мы имеем

$P(B) P(A|B) = P(A) P(B|A)$

Вот один способ интерпретировать это: если вы хотите проверить $A$ и $B$, вы можете сделать это в любом порядке:

1. вы можете сначала проверить $B$, затем $A$ при условии, что $B$, или

2. вы можете сначала проверить $A$, затем $B$ при условии, что $A$.

Чтобы попробовать, я вычислю долю молодых строителей двумя способами:

In [32]:
prob(young) * conditional(builder, young)

0.012314871170622844

In [33]:
prob(builder) * conditional(young, builder)

0.012314871170622844

То же самое!

**Упражнение:** Рассчитайте вероятность быть мужчиной-банкиром двумя способами и посмотрите, получите ли вы то же самое.

In [34]:
# Решение здесь

## Теорема 3

В предыдущем разделе мы установили, что

$P(B) P(A|B) = P(A) P(B|A)$

Если разделить на $P(B)$, получим Теорему 3:

$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$

И это, друзья мои, **теорема Байеса**.

Чтобы увидеть, как это работает, попробуем еще одну комбинацию наших утверждений. 

Давайте вычислим долю либеральных строителей, сначала используя функцию `conditional`:

In [35]:
conditional(liberal, builder)

0.24431625381744146

Теперь, используя теорему Байеса:

In [36]:
prob(liberal) * conditional(builder, liberal) / prob(builder)

0.24431625381744151

То же самое!

**Упражнение:** Попробуйте сами! Вычислите долю молодых людей, которые являются республиканцами, двумя способами: используя функцию `conditional` и теорему Байеса. Посмотрите, получите ли вы то же самое.

In [37]:
conditional(republican, young)

0.23319415448851774

In [38]:
prob(republican) * conditional(young, republican) / prob(young)

0.2331941544885177

## Резюме

Вот что у нас есть на данный момент:

**Теорема 1** дает нам новый способ вычисления условной вероятности с помощью конъюнкции:

$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$ 

**Теорема 2** дает нам новый способ вычисления конъюнкции с использованием условной вероятности:

$P(A~\mathrm{and}~B) = P(B) P(A|B)$

**Теорема 3**, также известная как теорема Байеса, дает нам способ перейти от $P(A|B)$ к $P(B|A)$ или наоборот:

$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$

Но тут вы можете спросить: "И что?" Если у нас есть все данные, мы можем вычислить любую желаемую вероятность, любую конъюнкцию или любую условную вероятность, просто подсчитав. Зачем нужны эти формулы?

И вы правы, *если* у нас есть все данные. Но часто мы этого не делаем, и в этом случае эти формулы могут быть очень полезны - особенно теорема Байеса.

В следующей записной книжке мы увидим, как это сделать.

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>